In [29]:
import os
import pandas as pd

folder = "/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/IN Carta [EB]/Results CSVs/11Plates_EDA"

In [11]:

csv_dict = {}
for root, dirs, files in os.walk(folder):
    for dir_name in dirs:
        csv_dict[dir_name] = {}
        folder_path = os.path.join(root, dir_name)
        for sub_root, sub_dirs, sub_files in os.walk(folder_path):
            for file_name in sub_files:
                file_path = os.path.join(folder_path, file_name)
                if "Red" in file_name:
                    name = "red"
                elif "Green" in file_name:
                    name = "green"
                if "Cell" in file_name:
                    name += "_cells"
                elif "Nuclei" in file_name:
                    name += "_nuclei"
                csv_dict[dir_name][name] = pd.read_csv(file_path)
            
# csv_dict

In [12]:
# drop unnecessary columns
columns_to_drop = [0, 4, 5]
for plate in csv_dict:
    for result  in csv_dict[plate]:
        df = csv_dict[plate][result]
        df.drop(df.columns[columns_to_drop], axis=1, inplace = True)
        if "nuclei" in result:
            df.drop(df.columns[[0, 1, 2]], axis=1, inplace = True)

In [19]:
# combine nuclei & cell data
combined_dict = {}
# combined_dict
for plate in csv_dict:
    combined_dict[plate] = {}
    combined_dict[plate]['green'] = pd.concat([csv_dict[plate]['green_cells'], csv_dict[plate]['green_nuclei']], axis=1)
    combined_dict[plate]['red'] = pd.concat([csv_dict[plate]['red_cells'], csv_dict[plate]['red_nuclei']], axis=1)

        

In [ ]:
for plate in combined_dict:
    for color in combined_dict[plate]:
        df = combined_dict[plate][color]
        df.to_csv(os.path.join('/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/IN Carta [EB]/Results CSVs/11Plates_EDA', plate, color + '_combined.csv'), index = False)


## Add New Plate Data to Dict
(might wanna change so that it doesn't rely on csv_dict being a saved item (just combined_dict would be)

In [30]:
# Find new subfolders to process
new_subfolders = []
for dir_name in next(os.walk(folder))[1]:
    combined_green_path = os.path.join(folder, dir_name, 'green_combined.csv')
    combined_red_path = os.path.join(folder, dir_name, 'red_combined.csv')
    if dir_name not in csv_dict or not (os.path.exists(combined_green_path) and os.path.exists(combined_red_path)):
        new_subfolders.append(dir_name)

# Process new subfolders
for dir_name in new_subfolders:
    csv_dict[dir_name] = {}
    folder_path = os.path.join(folder, dir_name)
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if "Red" in file_name:
            name = "red"
        elif "Green" in file_name:
            name = "green"
        if "Cell" in file_name:
            name += "_cells"
        elif "Nuclei" in file_name:
            name += "_nuclei"
        csv_dict[dir_name][name] = pd.read_csv(file_path)

# Drop unnecessary columns and combine nuclei & cell data
columns_to_drop = [0, 4, 5]
for plate in new_subfolders:
    for result, df in csv_dict[plate].items():
        df.drop(df.columns[columns_to_drop], axis=1, inplace=True)
        if "nuclei" in result:
            df.drop(df.columns[[0, 1, 2]], axis=1, inplace=True)
    combined_dict[plate] = {
        'green': pd.concat([csv_dict[plate]['green_cells'], csv_dict[plate]['green_nuclei']], axis=1),
        'red': pd.concat([csv_dict[plate]['red_cells'], csv_dict[plate]['red_nuclei']], axis=1)
    }

# Save combined data to CSV only for new subfolders
for plate in new_subfolders:
    for color, df in combined_dict[plate].items():
        output_path = os.path.join(folder, plate, f"{color}_combined.csv")
        df.to_csv(output_path, index=False)

In [35]:
# remove non-numerical data & fix well name
def modify_string(s):
    return s[:1] + s[4:]

for plate in combined_dict:
    for result  in combined_dict[plate]:
        df = combined_dict[plate][result]
        df.drop(df.columns[[1, 2]], axis=1, inplace = True)
        df['WELL LABEL'] = df['WELL LABEL'].apply(modify_string)
        df.set_index('WELL LABEL', inplace=True)


In [37]:
# save dict
import pickle

# Save to a pickle file
with open('/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/IN Carta [EB]/Results CSVs/11Plates_EDA/nuclei_and_cell_combined_dict.pkl', 'wb') as f:
    pickle.dump(combined_dict, f)

# Z-score Standardization

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [38]:
red_data = combined_dict['NIH-3']['red']
green_data = combined_dict['NIH-3']['green']

# Z-score Standardization
scaler_standard = StandardScaler()
red_data_standard = pd.DataFrame(scaler_standard.fit_transform(red_data), columns=red_data.columns)
green_data_standard = pd.DataFrame(scaler_standard.fit_transform(green_data), columns=green_data.columns)



<bound method NDFrame.head of      Cells Count  Cells Area_MEAN  Cells Area_SUM  Cells Area_SD  \
0       0.767296        -0.122517        0.572340      -0.462682   
1      -0.174361         0.190308       -0.156393      -0.522266   
2      -0.271296         0.091083       -0.293666       0.776073   
3      -0.042806         0.522231        0.127364      -0.281786   
4       0.102597         1.068048        0.545023       0.741802   
..           ...              ...             ...            ...   
379     0.933471         0.471239        1.095369       0.384294   
380    -0.873679         0.053591       -0.882578      -0.965940   
381     1.286592         0.073391        1.184474       0.098344   
382    -2.570046        -2.777859       -2.704028      -1.722159   
383    -2.784688        -3.027654       -2.822900      -2.260452   

     Cells Gyration Radius_MEAN  Cells Gyration Radius_SUM  \
0                     -0.192294                   0.636541   
1                      0.2648

In [ ]:

# Calculate Z-scores to identify outliers
z_scores_red = np.abs((red_data - red_data.mean()) / red_data.std())
z_scores_green = np.abs((green_data - green_data.mean()) / green_data.std())

# Identify outliers (Z-score threshold, e.g., |z| > 3)
outliers_red = (z_scores_red > 3).any(axis=1)
outliers_green = (z_scores_green > 3).any(axis=1)

# Visualize the outliers
plt.figure(figsize=(10, 6))
plt.scatter(range(len(z_scores_red)), z_scores_red.mean(axis=1), c=outliers_red, cmap='coolwarm', edgecolors='k')
plt.title('Red Cell Data Z-scores (Mean Z-score per Well)')
plt.xlabel('Well Index')
plt.ylabel('Mean Z-score')
plt.show()

plt.figure(figsize=(10, 6))
plt.scatter(range(len(z_scores_green)), z_scores_green.mean(axis=1), c=outliers_green, cmap='coolwarm', edgecolors='k')
plt.title('Green Cell Data Z-scores (Mean Z-score per Well)')
plt.xlabel('Well Index')
plt.ylabel('Mean Z-score')
plt.show()

print(f'Number of outlier wells in red cell data: {outliers_red.sum()}')
print(f'Number of outlier wells in green cell data: {outliers_green.sum()}')
